# Step-by-Step Warehouse Optimization Model Training

This notebook guides you through the process of training Machine Learning models for 3D warehouse packing. 
We will cover:
1.  **Setup**: Installing dependencies and cloning the repo (or assuming files are uploaded).
2.  **Data Generation**: Creating synthetic packing scenarios (Heuristic Labeling).
3.  **Model Training**: Training Neural Networks to mimic these strategies.
4.  **Evaluation**: Validating the models with metrics.
5.  **Download**: Getting the trained models.

**Note**: If running in Colab, ensure you upload the project files or clone your repository.

## 1. Setup Environment

In [ ]:
# Install dependencies
!pip install torch numpy pandas flask flask-cors scikit-learn

In [ ]:
import os
import torch

# Check CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create directories
os.makedirs('training_data', exist_ok=True)
os.makedirs('models', exist_ok=True)

## 1.5 Prepare Dataset & Train GAN

First, we convert the raw `bed-bpp` JSON dataset into a CSV format suitable for training the Generative Adversarial Network (GAN). Then, we train the GAN to learn the distributions of real-world items.

In [ ]:
!python tools/convert_dataset.py
!python gan/train.py

## 2. Generating Training Data (Labeling)

We need to create a dataset of "good" packing solutions. We use existing heuristic algorithms (Genetic Algorithm, Extremal Optimization) to solve thousands of random packing problems. The resulting $(x, y, z)$ coordinates become the labels our neural network will learn to predict.

*Note: This step can take a while (10-30 mins) depending on sample count.*

In [ ]:
# Import your project modules
# Ideally, you should have uploaded generate_training_data.py, optimizer.py, etc.
# If not, this cell assumes you are in the project root.

import random
import csv
import uuid
from optimizer import GeneticAlgorithm, ExtremalOptimization, HybridOptimizer

# Configuration for Colab speed
OUTPUT_DIR = "training_data"
SAMPLES_PER_ALGO = 50  # Generate 50 scenarios per algo
ITEMS_PER_SAMPLE = 20  


import sys
if './gan' not in sys.path:
    sys.path.append('./gan')
import pickle
import random
try:
    from model import Generator
except ImportError:
    from gan.model import Generator

def generate_gan_items(count, warehouse_dims):
    l_wh, w_wh, h_wh = warehouse_dims
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    scaler_path = 'gan/scaler.pkl'
    ckpt_path = 'gan/checkpoints/generator.pth'
    
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
        
    model = Generator(100, 4).to(device)
    model.load_state_dict(torch.load(ckpt_path, map_location=device))
    model.eval()
    
    z = torch.randn(count, 100).to(device)
    with torch.no_grad():
        gen_data = model(z).cpu().numpy()
        
    original_data = scaler.inverse_transform(gen_data)
    
    items = []
    categories = ['General', 'Electronics', 'Clothing']
    fragile_categories = {'Electronics'}
    
    for i in range(count):
        l, w, h, weight = original_data[i]
        l, w, h, weight = abs(l)*2.0, abs(w)*2.0, abs(h)*2.0, abs(weight)*2.0
        
        cat = random.choice(categories)
        is_fragile = 1 if cat in fragile_categories else 0
        is_stackable = 0 if is_fragile else (1 if random.random() > 0.1 else 0)
        can_rotate = 0 if h > 2 * min(l, w) else 1
        
        items.append({
            'id': str(uuid.uuid4()),
            'length': round(float(l), 2), 'width': round(float(w), 2), 'height': round(float(h), 2),
            'weight': round(float(weight), 2), 'category': cat,
            'can_rotate': can_rotate, 'stackable': is_stackable, 'fragility': is_fragile, 'access_freq': random.randint(1,10),
            'x': 0, 'y': 0, 'z': 0, 'rotation': 0
        })
    return items

def run_data_generation():
    print("Starting Data Generation...")
    algorithms = [
        ('fit_eo', ExtremalOptimization(iterations=20)),
        ('fit_ga', GeneticAlgorithm(population_size=20, generations=10)),
        ('fit_eo_ga', HybridOptimizer(ga_generations=1, eo_iterations=2)),
        ('fit_ga_eo', HybridOptimizer(ga_generations=5, eo_iterations=10))
    ]
    
    weights = {'space': 0.6, 'accessibility': 0.1, 'stability': 0.3}

    for algo_name, optimizer in algorithms:
        print(f"Generating for {algo_name}...")
        csv_path = os.path.join(OUTPUT_DIR, f"{algo_name}.csv")
        
        with open(csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            header = ['item_l', 'item_w', 'item_h', 'weight', 'fragile', 'stackable', 'can_rotate',
                      'wh_l', 'wh_w', 'wh_h', 'target_x', 'target_y', 'target_z', 'target_rot']
            writer.writerow(header)
            
            for _ in range(SAMPLES_PER_ALGO):
                # Randomize Warehouse Size
                wh_l = round(random.uniform(10, 30), 1)
                wh_w = round(random.uniform(10, 30), 1)
                wh_h = round(random.uniform(5, 15), 1)
                
                warehouse = {'id': 1, 'length': wh_l, 'width': wh_w, 'height': wh_h}
                items = generate_gan_items(ITEMS_PER_SAMPLE, (wh_l, wh_w, wh_h))
                
                try:
                    if 'Hybrid' in str(type(optimizer)) and algo_name == 'fit_eo_ga':
                        sol, _, _ = optimizer.optimize_eo_ga(items, warehouse, weights)
                    else:
                        sol, _, _ = optimizer.optimize(items, warehouse, weights)
                    
                    # Write valid placements
                    for idx, sol_item in enumerate(sol):
                        item = items[idx]
                        writer.writerow([
                            item['length'], item['width'], item['height'], item['weight'], 
                            item['fragility'], item['stackable'], item['can_rotate'],
                            wh_l, wh_w, wh_h,
                            sol_item['x'], sol_item['y'], sol_item['z'], sol_item['rotation']
                        ])
                except Exception as e:
                    print(f"Skipping sample error: {e}")
                    
    print("Data Generation Complete.")

run_data_generation()

## 3. Train Models

Now we train 4 separate Neural Networks (one for each strategy) using the CSV data we just generated.

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import glob

# Define Model Architecture (Same as project ml_utils.py)
class PackingModel(nn.Module):
    def __init__(self, input_dim=10, output_dim=4):
        super(PackingModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        out = self.fc4(x)
        return out

# Dataset Wrapper
class WarehouseDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        # Inputs: item dims (3) + weight + flags (3) + wh dims (3) = 10
        self.x = self.data.iloc[:, 0:10].values.astype('float32')
        # Targets: x, y, z, rot
        self.y = self.data.iloc[:, 10:14].values.astype('float32')
        
        # Normalize Inputs (Approximate)
        self.x[:, 0:3] /= 10.0  # Item dims
        self.x[:, 7:10] /= 50.0 # Warehouse dims
        
        # Normalize Targets (Relative to Warehouse)
        wh_l = self.data['wh_l'].values.astype('float32')
        wh_w = self.data['wh_w'].values.astype('float32')
        wh_h = self.data['wh_h'].values.astype('float32')
        
        self.y[:, 0] /= (wh_l + 1e-5)
        self.y[:, 1] /= (wh_w + 1e-5)
        self.y[:, 2] /= (wh_h + 1e-5)
        self.y[:, 3] /= 6.0 # Rotation normalized

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])

def train_all_models():
    csv_files = glob.glob(os.path.join(OUTPUT_DIR, "*.csv"))
    
    for csv_file in csv_files:
        model_name = os.path.basename(csv_file).replace('.csv', '')
        print(f"\nTraining model: {model_name}...")
        
        dataset = WarehouseDataset(csv_file)
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
        
        model = PackingModel().to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        for epoch in range(20): # 20 Epochs for demo
            total_loss = 0
            for bx, by in dataloader:
                bx, by = bx.to(device), by.to(device)
                optimizer.zero_grad()
                pred = model(bx)
                loss = criterion(pred, by)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            
            if (epoch+1) % 5 == 0:
                print(f"Epoch {epoch+1}/20 - Loss: {total_loss/len(dataloader):.6f}")
        
        # Save
        save_path = os.path.join("models", f"model_{model_name}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"Saved to {save_path}")

train_all_models()

## 4. Evaluation and Metrics

We evaluate the trained models by running inference on test packing scenarios and calculating key performance indicators: Space Utilization, Accessibility, and Stability.

In [ ]:
import numpy as np
import torch
import os

# Simple Physics/Metrics Logic (Simplified from optimizer.py)
def calculate_metrics(solution, items, wh_dims):
    # solution: list of dicts {x, y, z, rotation}
    # items: list of dicts
    
    # 1. Space Utilization
    total_item_vol = sum(i['length'] * i['width'] * i['height'] for i in items)
    wh_vol = wh_dims[0] * wh_dims[1] * wh_dims[2]
    space_util = total_item_vol / wh_vol
    
    # 2. Stability (Simplified: check if z > 0 has support)
    stable_count = 0
    for i, sol in enumerate(solution):
        if sol['z'] <= 0.01: # On floor
            stable_count += 1
        else:
            # Check if supported by another item
            supported = False
            for j, other in enumerate(solution):
                if i == j: continue
                if abs(other['z'] + items[j]['height'] - sol['z']) < 0.05: # Vertical contact
                    # Check horizontal overlap
                    if (abs(sol['x'] - other['x']) < (items[i]['length'] + items[j]['length'])/2 and
                        abs(sol['y'] - other['y']) < (items[i]['width'] + items[j]['width'])/2):
                        supported = True
                        break
            if supported:
                stable_count += 1
    stability = stable_count / len(solution)
    
    # 3. Accessibility (Distance to door at 0,0)
    dists = [np.sqrt(s['x']**2 + s['y']**2) for s in solution]
    accessibility = 1.0 / (1.0 + np.mean(dists))
    
    return space_util, accessibility, stability

def evaluate_model(model_name):
    print(f"\nEvaluating {model_name}...")
    model_path = os.path.join("models", f"{model_name}.pth")
    if not os.path.exists(model_path):
        print("Model not found.")
        return
        
    model = PackingModel().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    # Generate Test Case
    wh_l, wh_w, wh_h = 20.0, 20.0, 10.0
    items = generate_gan_items(20, (wh_l, wh_w, wh_h))
    
    # Prepare Input
    features = []
    for item in items:
        features.append([
            item['length']/10.0, item['width']/10.0, item['height']/10.0,
            item['weight']/50.0, item['fragility'], item['stackable'], item['can_rotate'],
            wh_l/50.0, wh_w/50.0, wh_h/50.0
        ])
    
    inputs = torch.tensor(features, dtype=torch.float32).to(device)
    
    with torch.no_grad():
        outputs = model(inputs).cpu().numpy()
        
    # Decode Output
    solution = []
    for i, row in enumerate(outputs):
        solution.append({
            'x': row[0] * wh_l,
            'y': row[1] * wh_w,
            'z': max(0, row[2] * wh_h), # Floor clamp
            'rotation': int(row[3] * 6)
        })
        
    # Calculate Metrics
    sp, acc, stb = calculate_metrics(solution, items, (wh_l, wh_w, wh_h))
    print(f"  Space Utilization: {sp*100:.2f}%")
    print(f"  Stability Score:   {stb*100:.2f}%")
    print(f"  Accessibility:     {acc:.4f}")

# Run Eval
evaluate_model('model_fit_ga')
evaluate_model('model_fit_eo')
evaluate_model('model_fit_eo_ga')
evaluate_model('model_fit_ga_eo')


## 5. Download Models
Zip the trained models so you can download them and use them in your local Flask app.

In [ ]:
!zip -r trained_models.zip models

try:
    from google.colab import files
    files.download('trained_models.zip')
except ImportError:
    print("Not running in Colab environment. Files are in 'models/' directory.")